PARCIAL 2 

Ejercicio 1: Configuración del Entorno y Carga de Modelo Base
Objetivo
Establecer el entorno de desarrollo necesario para trabajar con modelos LLM y cargar un modelo pre-entrenado utilizando las bibliotecas Transformers y PyTorch.

Código Base para Completar

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from pathlib import Path

In [18]:
# Configurar y crear automáticamente la caché de modelos
cache_dir = os.path.join(os.path.expanduser('~'), 'cache_modelos_hf')
os.environ['HF_HOME'] = cache_dir
Path(cache_dir).mkdir(parents=True, exist_ok=True)
print(f"\nCaché de modelos configurada en: {cache_dir}")

def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.
    """
    try:
        # Configurar tokenizador
        tokenizador = AutoTokenizer.from_pretrained(nombre_modelo, cache_dir=cache_dir)
        tokenizador.pad_token = tokenizador.eos_token
        
        # Cargar modelo con manejo de formato TensorFlow/PyTorch
        try:
            modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo, cache_dir=cache_dir)
        except:
            print("Intentando cargar con pesos de TensorFlow...")
            modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo, cache_dir=cache_dir, from_tf=True)
        
        modelo.eval()
        
        # Optimizar para GPU
        if torch.cuda.is_available():
            modelo = modelo.half().to('cuda')
            
        return modelo, tokenizador
        
    except Exception as e:
        print(f"\nError al cargar el modelo: {str(e)}")
        return None, None

def verificar_dispositivo():
    """Verifica y muestra información del dispositivo disponible."""
    dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    if dispositivo.type == 'cuda':
        print("\n[+] GPU Detectada:")
        print(f"  - Nombre: {torch.cuda.get_device_name(0)}")
        print(f"  - Memoria: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f} GB")
    
    return dispositivo

def main():
    print("\n=== Configuración Inicial ===")
    dispositivo = verificar_dispositivo()
    
    # Seleccionar modelo según hardware
    modelo_nombre = "gpt2"
    print(f"\n[+] Cargando modelo: {modelo_nombre}")
    
    modelo, tokenizador = cargar_modelo(modelo_nombre)
    if modelo is None:
        print("\nProbando con una versión alternativa del modelo...")
        modelo_nombre = "distilgpt2"  # Versión más ligera alternativa
        modelo, tokenizador = cargar_modelo(modelo_nombre)
        if modelo is None:
            return

    # Configuración de generación
    config_gen = {
        'max_new_tokens': 100,
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.9,
        'repetition_penalty': 1.2,
        'pad_token_id': tokenizador.eos_token_id
    }

    print("\nSistema listo. Escribe tu mensaje (o 'salir' para terminar)")
    while True:
        try:
            prompt = input("\n[Tu mensaje]: ").strip()
            if prompt.lower() in ['salir', 'exit', 'quit']:
                break
                
            inputs = tokenizador(prompt, return_tensors='pt').to(dispositivo)
            
            with torch.no_grad():
                salida = modelo.generate(**inputs, **config_gen)
                
            respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
            print(f"\n[Respuesta]: {respuesta[len(prompt):].lstrip()}")
            
        except KeyboardInterrupt:
            print("\nOperación cancelada")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")

if __name__ == "__main__":
    main()


Caché de modelos configurada en: C:\Users\Dani\cache_modelos_hf

=== Configuración Inicial ===

[+] Cargando modelo: gpt2
Intentando cargar con pesos de TensorFlow...

Error al cargar el modelo: gpt2 does not appear to have a file named pytorch_model.bin but there is a file for TensorFlow weights. Use `from_tf=True` to load this model from those weights.

Probando con una versión alternativa del modelo...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Intentando cargar con pesos de TensorFlow...

Error al cargar el modelo: distilgpt2 does not appear to have a file named pytorch_model.bin but there is a file for TensorFlow weights. Use `from_tf=True` to load this model from those weights.


In [ ]:
# 2 INTENTOS

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# TODO: Configurar las variables de entorno para la caché de modelos
# Establecer la carpeta donde se almacenarán los modelos descargados
os.environ['TRANSFORMERS_CACHE'] = './cache_modelos'
print(f"La caché de modelos se guardará en: {os.environ['TRANSFORMERS_CACHE']}")

def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.

    Args:
        nombre_modelo (str): Identificador del modelo en Hugging Face Hub
    Returns:
        tuple: (modelo, tokenizador)
    """
    print(f"Cargando modelo: {nombre_modelo}...")
    # TODO: Implementar la carga del modelo y tokenizador
    # Utiliza AutoModelForCausalLM y AutoTokenizer
    try:
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
        tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
        print(f"Modelo {nombre_modelo} y tokenizador cargados exitosamente.")
    except Exception as e:
        print(f"Error al cargar el modelo {nombre_modelo}: {e}")
        return None, None

    # TODO: Configurar el modelo para inferencia (evaluar y usar half-precision si es posible)
    modelo.eval()
    if torch.cuda.is_available():
        modelo = modelo.half().cuda()
        print("Modelo movido a GPU y utilizando half-precision (float16).")
    else:
        print("Utilizando CPU para la inferencia.")

    return modelo, tokenizador

def verificar_dispositivo():
    """
    Verifica el dispositivo disponible (CPU/GPU) y muestra información relevante.

    Returns:
        torch.device: Dispositivo a utilizar
    """
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")
        # TODO: Si hay GPU disponible, mostrar información sobre la misma
        print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
        print(f"Cantidad de GPUs disponibles: {torch.cuda.device_count()}")
    else:
        dispositivo = torch.device("cpu")
        print("No se encontró GPU disponible, utilizando CPU.")
    return dispositivo

# Función principal de prueba
def main():
    dispositivo = verificar_dispositivo()
    print(f"Utilizando dispositivo: {dispositivo}")

    # TODO: Cargar un modelo pequeño adecuado para chatbots (ej. Mistral-7B, GPT2, etc.)
    nombre_modelo = "gpt2"  # Un modelo pequeño para pruebas
    modelo, tokenizador = cargar_modelo(nombre_modelo)

    if modelo is not None and tokenizador is not None:
        # TODO: Realizar una prueba simple de generación de texto
        texto_prueba = "Hola, ¿cómo estás?"
        input_ids = tokenizador.encode(texto_prueba, return_tensors="pt").to(dispositivo)

        with torch.no_grad():
            output = modelo.generate(input_ids, max_length=50, num_return_sequences=1)

        respuesta = tokenizador.decode(output[0], skip_special_tokens=True)
        print(f"Pregunta: {texto_prueba}")
        print(f"Respuesta generada: {respuesta}")

if __name__ == "__main__":
    main()

La caché de modelos se guardará en: ./cache_modelos
No se encontró GPU disponible, utilizando CPU.
Utilizando dispositivo: cpu
Cargando modelo: gpt2...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Modelo gpt2 y tokenizador cargados exitosamente.
Utilizando CPU para la inferencia.
Pregunta: Hola, ¿cómo estás?
Respuesta generada: Hola, ¿cómo estás?

I'm sorry, I'm sorry.

I'm sorry, I'm sorry.

I'm sorry, I'm sorry.

I'm sorry, I'm sorry


Ejercicio 2: Procesamiento de Entrada y Generación de Respuestas

Objetivo: Desarrollar las funciones necesarias para procesar la entrada del usuario, preparar los tokens para el modelo y generar respuestas coherentes.

Código Base para Completar

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# TODO: Configurar las variables de entorno para la caché de modelos
# Establecer la carpeta donde se almacenarán los modelos descargados
os.environ['TRANSFORMERS_CACHE'] = './cache_modelos'
print(f"La caché de modelos se guardará en: {os.environ['TRANSFORMERS_CACHE']}")

def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.

    Args:
        nombre_modelo (str): Identificador del modelo en Hugging Face Hub

    Returns:
        tuple: (modelo, tokenizador)
    """
    # TODO: Implementar la carga del modelo y tokenizador
    # Utiliza AutoModelForCausalLM y AutoTokenizer
    try:
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
        tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
        print(f"Modelo {nombre_modelo} y tokenizador cargados exitosamente.")
    except Exception as e:
        print(f"Error al cargar el modelo {nombre_modelo}: {e}")
        return None, None

    # TODO: Configurar el modelo para inferencia (evaluar y usar half-precision si es posible)
    modelo.eval()
    if torch.cuda.is_available():
        modelo = modelo.half().cuda()
        print("Modelo movido a GPU y utilizando half-precision (float16).")
    else:
        print("Utilizando CPU para la inferencia.")

    return modelo, tokenizador

def verificar_dispositivo():
    """
    Verifica el dispositivo disponible (CPU/GPU) y muestra información relevante.

    Returns:
        torch.device: Dispositivo a utilizar
    """
    # TODO: Implementar la detección del dispositivo
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")
        # TODO: Si hay GPU disponible, mostrar información sobre la misma
        print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
        print(f"Cantidad de GPUs disponibles: {torch.cuda.device_count()}")
    else:
        dispositivo = torch.device("cpu")
        print("No se encontró GPU disponible, utilizando CPU.")
    return dispositivo

# Función principal de prueba
def main():
    dispositivo = verificar_dispositivo()
    print(f"Utilizando dispositivo: {dispositivo}")

    # TODO: Cargar un modelo pequeño adecuado para chatbots (ej. Mistral-7B, GPT2, etc.)
    nombre_modelo = "gpt2"  # Un modelo pequeño para pruebas
    modelo, tokenizador = cargar_modelo(nombre_modelo)

    if modelo is not None and tokenizador is not None:
        # TODO: Realizar una prueba simple de generación de texto
        texto_prueba = "Hola, ¿cómo estás?"
        entrada_tokenizada = tokenizador(texto_prueba, return_tensors="pt")
        input_ids = entrada_tokenizada.input_ids.to(dispositivo)
        attention_mask = entrada_tokenizada.attention_mask.to(dispositivo)

        with torch.no_grad():
            output = modelo.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_return_sequences=1,
                temperature=0.8,
                top_p=0.9,
                do_sample=True  # Habilitar el muestreo para usar temperature y top_p
            )

        respuesta = tokenizador.decode(output[0], skip_special_tokens=True)
        print(f"Pregunta: {texto_prueba}")
        print(f"Respuesta generada: {respuesta}")

if __name__ == "__main__":
    main()

La caché de modelos se guardará en: ./cache_modelos
No se encontró GPU disponible, utilizando CPU.
Utilizando dispositivo: cpu


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Modelo gpt2 y tokenizador cargados exitosamente.
Utilizando CPU para la inferencia.
Pregunta: Hola, ¿cómo estás?
Respuesta generada: Hola, ¿cómo estás? O usted toda?

(Cómo, á vías.)

O, where are you?

(Cómo, á vías


Ejercicio 3: Manejo de Contexto Conversacional

Objetivo:
Implementar un sistema para mantener el contexto de la conversación, permitiendo al chatbot recordar intercambios anteriores y responder coherentemente a conversaciones prolongadas.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def cargar_modelo(modelo_id):
    """Carga el modelo y el tokenizador desde Hugging Face."""
    tokenizador = AutoTokenizer.from_pretrained(modelo_id)
    modelo = AutoModelForCausalLM.from_pretrained(modelo_id)
    return modelo, tokenizador

def verificar_dispositivo():
    """Verifica y devuelve el dispositivo a usar (CPU o GPU)."""
    return "cuda" if torch.cuda.is_available() else "cpu"

class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """

    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        """
        Inicializa el gestor de contexto.

        Args:
            longitud_maxima (int): Número máximo de tokens a mantener en el contexto
            formato_mensaje (callable): Función para formatear mensajes (por defecto, None)
        """
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        """
        Formato predeterminado para mensajes.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje

        Returns:
            str: Mensaje formateado
        """
        if rol == "sistema":
            return f"<<SYSTEM>> {contenido} <</SYSTEM>>"
        elif rol == "usuario":
            return f"<<USER>> {contenido} <</USER>>"
        elif rol == "asistente":
            return f"<<ASSISTANT>> {contenido} <</ASSISTANT>>"
        return f"{rol}: {contenido}"

    def agregar_mensaje(self, rol, contenido):
        """
        Agrega un mensaje al historial de conversación.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje
        """
        self.historial.append({"rol": rol, "contenido": contenido})

    def construir_prompt_completo(self):
        """
        Construye un prompt completo basado en el historial.

        Returns:
            str: Prompt completo para el modelo
        """
        mensajes_formateados = [self.formato_mensaje(msg["rol"], msg["contenido"]) for msg in self.historial]
        return "\n".join(mensajes_formateados)

    def truncar_historial(self, tokenizador):
        """
        Trunca el historial si excede la longitud máxima.

        Args:
            tokenizador: Tokenizador del modelo
        """
        total_tokens = 0
        indices_a_eliminar = []

        # Calcular el número total de tokens en el historial
        for i, mensaje in enumerate(reversed(self.historial)):
            tokens = tokenizador.encode(self.formato_mensaje(mensaje["rol"], mensaje["contenido"]), add_special_tokens=False)
            total_tokens += len(tokens)
            if total_tokens > self.longitud_maxima:
                indices_a_eliminar.append(len(self.historial) - 1 - i)

        # Eliminar los mensajes más antiguos si se excede la longitud máxima
        if indices_a_eliminar:
            indices_a_eliminar.reverse()  # Eliminar en orden ascendente para no afectar los índices
            nuevo_historial = [msg for i, msg in enumerate(self.historial) if i not in indices_a_eliminar]
            self.historial = nuevo_historial

        # Consideración adicional: Podrías implementar una lógica más compleja aquí,
        # como resumir mensajes antiguos en lugar de simplemente eliminarlos,
        # o asegurarte de mantener siempre el mensaje del sistema.

class Chatbot:
    """
    Implementación de chatbot con manejo de contexto.
    """

    def __init__(self, modelo_id, instrucciones_sistema=None):
        """
        Inicializa el chatbot.

        Args:
            modelo_id (str): Identificador del modelo en Hugging Face
            instrucciones_sistema (str): Instrucciones de comportamiento del sistema
        """
        self.modelo, self.tokenizador = cargar_modelo(modelo_id)
        self.dispositivo = verificar_dispositivo()
        self.gestor_contexto = GestorContexto()

        # Inicializar el contexto con instrucciones del sistema
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("sistema", instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta al mensaje del usuario.

        Args:
            mensaje_usuario (str): Mensaje del usuario
            parametros_generacion (dict): Parámetros para la generación

        Returns:
            str: Respuesta del chatbot
        """
        # 1. Agregar mensaje del usuario al contexto
        self.gestor_contexto.agregar_mensaje("usuario", mensaje_usuario)

        # 2. Construir el prompt completo
        prompt = self.gestor_contexto.construir_prompt_completo()

        # 3. Preprocesar la entrada
        inputs = self.tokenizador(prompt, return_tensors="pt").to(self.dispositivo)

        # 4. Generar la respuesta
        parametros_generacion = parametros_generacion if parametros_generacion else {
            "max_length": 150,
            "num_beams": 5,
            "no_repeat_ngram_size": 2,
            "early_stopping": True,
        }
        with torch.no_grad():
            output = self.modelo.generate(**inputs, **parametros_generacion)

        # 5. Decodificar la respuesta y agregarla al contexto
        respuesta = self.tokenizador.decode(output[0], skip_special_tokens=True)
        # Considerar cómo extraer solo la parte de la respuesta del asistente
        # si el modelo genera el prompt completo de vuelta.
        respuesta_limpia = respuesta.replace(prompt, "").strip()
        self.gestor_contexto.agregar_mensaje("asistente", respuesta_limpia)

        # 6. Truncar el historial si es necesario
        self.gestor_contexto.truncar_historial(self.tokenizador)

        return respuesta_limpia

def prueba_conversacion():
    # Crear una instancia del chatbot
    modelo_id = "gpt2"  # Puedes probar con un modelo más pequeño para empezar
    instrucciones_sistema = "Eres un asistente amigable y servicial."
    chatbot = Chatbot(modelo_id, instrucciones_sistema)

    print(f"Chatbot inicializado con el modelo: {modelo_id}")
    print(f"Instrucciones del sistema: {instrucciones_sistema}\n")

    # Simular una conversación de varios turnos
    while True:
        mensaje_usuario = input("Usuario: ")
        if mensaje_usuario.lower() == "salir":
            break

        respuesta = chatbot.responder(mensaje_usuario)
        print(f"Asistente: {respuesta}\n")

if __name__ == "__main__":
    prueba_conversacion()

Chatbot inicializado con el modelo: gpt2
Instrucciones del sistema: Eres un asistente amigable y servicial.



In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """
    
    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        """
        Inicializa el gestor de contexto.
        
        Args:
            longitud_maxima (int): Número máximo de tokens a mantener en el contexto.
            formato_mensaje (callable): Función para formatear mensajes (por defecto, None).
        """
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado
        
    def _formato_predeterminado(self, rol, contenido):
        """
        Formato predeterminado para mensajes.
        
        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'.
            contenido (str): Contenido del mensaje.
            
        Returns:
            str: Mensaje formateado.
        """
        return f"{rol}: {contenido}"
    
    def agregar_mensaje(self, rol, contenido):
        """
        Agrega un mensaje al historial de conversación.
        
        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'.
            contenido (str): Contenido del mensaje.
        """
        mensaje = self.formato_mensaje(rol, contenido)
        self.historial.append(mensaje)
        self.truncar_historial()

    def construir_prompt_completo(self):
        """
        Construye un prompt completo basado en el historial.
        
        Returns:
            str: Prompt completo para el modelo.
        """
        return "\n".join(self.historial)
    
    def truncar_historial(self):
        """
        Trunca el historial si excede la longitud máxima.
        """
        while True:
            # Convertir el historial a tokens y verificar la longitud
            tokens = self.historial  # Aquí se debería usar un tokenizador real para convertir el texto a tokens
            if len(tokens) <= self.longitud_maxima:
                break
            # Eliminar el mensaje más antiguo
            self.historial.pop(0)
            
class Chatbot:
    """
    Implementación de chatbot con manejo de contexto.
    """
    
    def __init__(self, modelo_id, instrucciones_sistema=None):
        """
        Inicializa el chatbot.
        
        Args:
            modelo_id (str): Identificador del modelo en Hugging Face.
            instrucciones_sistema (str): Instrucciones de comportamiento del sistema.
        """
        self.tokenizador = GPT2Tokenizer.from_pretrained(modelo_id)
        self.modelo = GPT2LMHeadModel.from_pretrained(modelo_id)
        self.dispositivo = "cuda" if torch.cuda.is_available() else "cpu"
        self.modelo.to(self.dispositivo)
        
        self.gestor_contexto = GestorContexto()
        
        # Inicializar el contexto con instrucciones del sistema, si las hay
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje('sistema', instrucciones_sistema)
        
        # Establecer el token de padding para evitar problemas
        self.tokenizador.pad_token = self.tokenizador.eos_token
    
    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta al mensaje del usuario.
        
        Args:
            mensaje_usuario (str): Mensaje del usuario.
            parametros_generacion (dict): Parámetros para la generación.
            
        Returns:
            str: Respuesta del chatbot.
        """
        # Evitar agregar el mensaje del sistema repetidamente
        self.gestor_contexto.agregar_mensaje('usuario', mensaje_usuario)
        
        # Construir el prompt completo basado en el historial
        prompt_completo = self.gestor_contexto.construir_prompt_completo()
        
        # Preprocesar la entrada
        entrada_procesada = self.preprocesar_entrada(prompt_completo)
        
        # Generar la respuesta utilizando el modelo
        respuesta = self.generar_respuesta(entrada_procesada, parametros_generacion)
        
        # Agregar respuesta del asistente al contexto
        self.gestor_contexto.agregar_mensaje('asistente', respuesta)
        
        return respuesta
    
    def preprocesar_entrada(self, prompt_completo):
        """
        Preprocesa la entrada antes de pasarlo al modelo.
        
        Args:
            prompt_completo (str): El prompt completo construido.
        
        Returns:
            str: Entrada preprocesada.
        """
        return prompt_completo
    
    def generar_respuesta(self, entrada_procesada, parametros_generacion):
        """
        Genera la respuesta utilizando el modelo.
        
        Args:
            entrada_procesada (str): La entrada que será procesada por el modelo.
            parametros_generacion (dict): Parámetros para la generación.
        
        Returns:
            str: La respuesta generada por el modelo.
        """
        input_ids = self.tokenizador.encode(entrada_procesada, return_tensors="pt").to(self.dispositivo)
        
        # Crear la máscara de atención
        attention_mask = (input_ids != self.tokenizador.pad_token_id).type(torch.uint8).to(self.dispositivo)
        
        # Ajustar el valor de max_new_tokens si la entrada es demasiado larga
        max_new_tokens = 50  # Ajusta esto según sea necesario
        
        # Verificar si la longitud de la entrada excede el límite
        if input_ids.shape[1] > 1024:  # Suponiendo que el modelo tiene un límite de 1024 tokens
            input_ids = input_ids[:, -1024:]  # Recortar la entrada a los últimos 1024 tokens
        
        # Generación de la respuesta
        outputs = self.modelo.generate(input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, num_return_sequences=1)
        
        respuesta = self.tokenizador.decode(outputs[0], skip_special_tokens=True)
        
        return respuesta

def prueba_conversacion():
    # Crear una instancia del chatbot
    chatbot = Chatbot("gpt2", instrucciones_sistema="Este es un chatbot que recuerda los mensajes anteriores.")
    
    # Simular una conversación de varios turnos
    print("Usuario: Hola, ¿cómo estás?")
    respuesta = chatbot.responder("Hola, ¿cómo estás?")
    print("Asistente:", respuesta)
    
    print("\nUsuario: ¿Qué puedes hacer?")
    respuesta = chatbot.responder("¿Qué puedes hacer?")
    print("Asistente:", respuesta)
    
    print("\nUsuario: ¿Puedes recordar lo que hablamos antes?")
    respuesta = chatbot.responder("¿Puedes recordar lo que hablamos antes?")
    print("Asistente:", respuesta)

# Ejecutar la prueba de conversación
prueba_conversacion()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Usuario: Hola, ¿cómo estás?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás

Usuario: ¿Qué puedes hacer?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?
asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás
usuario: ¿Qué puedes hacer?

asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.

usuario: Hola, ¿cómo estás?

us

Usuario: ¿Puedes recordar lo que hablamos antes?
Asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?
asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás?

usuario: Hola, ¿cómo estás
usuario: ¿Qué puedes hacer?
asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: Hola, ¿cómo estás?
asistente: sistema: Este es un chatbot que recuerda los mensajes anteriores.
usuario: 

Ejercicio 4: Optimización del Modelo para Recursos Limitados

Objetivo:

Implementar técnicas de optimización para mejorar la velocidad de inferencia y reducir el consumo de memoria, permitiendo que el chatbot funcione eficientemente en dispositivos con recursos limitados.

Código Base para Completar

In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BitsAndBytesConfig
import torch
import time
import psutil

def configurar_cuantizacion(bits=4):
    """
    Configura los parámetros para la cuantización del modelo.
    
    Args:
        bits (int): Bits para cuantización (4 u 8)
    
    Returns:
        BitsAndBytesConfig: Configuración de cuantización
    """
    if bits not in [4, 8]:
        raise ValueError("Solo se permite cuantización de 4 o 8 bits")
    
    # Configurar la cuantización con BitsAndBytesConfig
    config_cuantizacion = BitsAndBytesConfig(
        load_in_8bit=bits == 8,  # Usar 8 bits si 'bits' es 8
        load_in_4bit=bits == 4   # Usar 4 bits si 'bits' es 4
    )
    
    return config_cuantizacion

def cargar_modelo_optimizado(nombre_modelo, optimizaciones=None):
    from transformers import AutoTokenizer, AutoModelForCausalLM

    if optimizaciones is None:
        optimizaciones = {
            "cuantizacion": True,
            "bits": 4,
            "offload_cpu": False,
            "flash_attention": False
        }

    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Cargar modelo con cuantización
    if optimizaciones["cuantizacion"]:
        if optimizaciones["bits"] == 4:
            modelo = AutoModelForCausalLM.from_pretrained(
                nombre_modelo,
                load_in_4bit=True,
                device_map="auto"
            )
        elif optimizaciones["bits"] == 8:
            modelo = AutoModelForCausalLM.from_pretrained(
                nombre_modelo,
                load_in_8bit=True,
                device_map="auto"
            )
        else:
            raise ValueError("Bits debe ser 4 u 8 para cuantización")
    else:
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)

    return modelo, tokenizador


def aplicar_sliding_window(modelo, window_size=1024):
    """
    Configura la atención de ventana deslizante para procesar secuencias largas.
    
    Args:
        modelo: Modelo a configurar
        window_size (int): Tamaño de la ventana de atención
    """
    # Este paso depende de si el modelo soporta la técnica de "sliding window".
    # Para algunos modelos, como Longformer, se puede aplicar esta técnica.
    # En GPT2, este tipo de atención no está disponible nativamente.
    pass

def evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo):
    """
    Evalúa el rendimiento del modelo en términos de velocidad y memoria.
    
    Args:
        modelo: Modelo a evaluar
        tokenizador: Tokenizador del modelo
        texto_prueba (str): Texto para pruebas de rendimiento
        dispositivo: Dispositivo donde se ejecutará
    
    Returns:
        dict: Métricas de rendimiento
    """
    # Medir tiempo de inferencia
    start_time = time.time()
    inputs = tokenizador(texto_prueba, return_tensors="pt").to(dispositivo)
    
    with torch.no_grad():
        modelo.eval()
        outputs = modelo(**inputs)
    
    end_time = time.time()
    
    # Calcular el uso de memoria
    memoria_inicial = psutil.virtual_memory().used
    memoria_final = psutil.virtual_memory().used
    uso_memoria = memoria_final - memoria_inicial  # en bytes
    
    # Calcular tokens por segundo
    tiempo_inferencia = end_time - start_time
    tokens = len(inputs["input_ids"][0])
    tokens_por_segundo = tokens / tiempo_inferencia if tiempo_inferencia > 0 else 0
    
    return {
        "tiempo_inferencia": tiempo_inferencia,
        "uso_memoria": uso_memoria,
        "tokens_por_segundo": tokens_por_segundo
    }

def demo_optimizaciones():
    """
    Demuestra el impacto de diferentes optimizaciones en el rendimiento.
    """
    # Texto de prueba
    texto_prueba = "Este es un texto de prueba para evaluar el rendimiento del modelo."

    # 1. Cargar el modelo base (sin optimizaciones)
    modelo_base, tokenizador_base = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": False})
    rendimiento_base = evaluar_rendimiento(modelo_base, tokenizador_base, texto_prueba, dispositivo="cpu")
    
    # 2. Cargar el modelo con cuantización de 4 bits
    modelo_cuantizado_4, tokenizador_cuantizado_4 = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": True, "bits": 4})
    rendimiento_cuantizado_4 = evaluar_rendimiento(modelo_cuantizado_4, tokenizador_cuantizado_4, texto_prueba, dispositivo="cpu")
    
    # 3. Cargar el modelo con atención de ventana deslizante
    modelo_con_sliding_window, tokenizador_sliding_window = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": False})
    aplicar_sliding_window(modelo_con_sliding_window, window_size=1024)
    rendimiento_sliding_window = evaluar_rendimiento(modelo_con_sliding_window, tokenizador_sliding_window, texto_prueba, dispositivo="cpu")
    
    # 4. Cargar el modelo con todas las optimizaciones
    modelo_optimizado, tokenizador_optimizado = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": True, "bits": 4, "offload_cpu": False, "flash_attention": True})
    rendimiento_optimizado = evaluar_rendimiento(modelo_optimizado, tokenizador_optimizado, texto_prueba, dispositivo="cpu")
    
    # Mostrar los resultados
    print("Rendimiento del modelo base:", rendimiento_base)
    print("Rendimiento del modelo con cuantización de 4 bits:", rendimiento_cuantizado_4)
    print("Rendimiento del modelo con sliding window:", rendimiento_sliding_window)
    print("Rendimiento del modelo con todas las optimizaciones:", rendimiento_optimizado)

# Ejecutar la demostración de optimizaciones
demo_optimizaciones()


ValueError: Using a `device_map` or `tp_plan` requires `accelerate`. You can install it with `pip install accelerate`

Ejercicio 5: Personalización del Chatbot y Despliegue

Objetivo:
Implementar técnicas para personalizar el comportamiento del chatbot y prepararlo para su despliegue como una aplicación web simple.

Código Base para Completar

In [ ]:
import gradio as gr
from peft import LoraConfig, get_peft_model, TaskType

def configurar_peft(modelo, r=8, lora_alpha=32):
    """
    Configura el modelo para fine-tuning con PEFT/LoRA.
    
    Args:
        modelo: Modelo base
        r (int): Rango de adaptadores LoRA
        lora_alpha (int): Escala alpha para LoRA
    
    Returns:
        modelo: Modelo adaptado para fine-tuning
    """
    # TODO: Implementar la configuración de PEFT
    # Crear LoraConfig y aplicarla al modelo
    # ...
    
    return modelo_peft

def guardar_modelo(modelo, tokenizador, ruta):
    """
    Guarda el modelo y tokenizador en una ruta específica.
    
    Args:
        modelo: Modelo a guardar
        tokenizador: Tokenizador del modelo
        ruta (str): Ruta donde guardar
    """
    # TODO: Implementar el guardado de modelo y tokenizador
    # ...

def cargar_modelo_personalizado(ruta):
    """
    Carga un modelo personalizado desde una ruta específica.
    
    Args:
        ruta (str): Ruta del modelo
        
    Returns:
        tuple: (modelo, tokenizador)
    """
    # TODO: Implementar la carga del modelo personalizado
    # ...
    
    return modelo, tokenizador

# Interfaz web simple con Gradio
def crear_interfaz_web(chatbot):
    """
    Crea una interfaz web simple para el chatbot usando Gradio.
    
    Args:
        chatbot: Instancia del chatbot
        
    Returns:
        gr.Interface: Interfaz de Gradio
    """
    # TODO: Implementar la interfaz con Gradio
    # Definir función de callback para procesar entradas y generar respuestas
    # ...
    
    # TODO: Configurar la interfaz con componentes adecuados
    # ...
    
    return interfaz

# Función principal para el despliegue
def main_despliegue():
    # TODO: Cargar el modelo personalizado
    # ...
    
    # TODO: Crear instancia del chatbot
    # ...
    
    # TODO: Crear y lanzar la interfaz web
    # ...
    
    # TODO: (Opcional) Configurar parámetros para el despliegue
    # ...

if __name__ == "__main__":
    main_despliegue()

# Preguntas Teóricas
## 1. ¿Cuáles son las diferencias fundamentales entre los modelos encoder-only, decoder-only y encoder-decoder en el contexto de los chatbots conversacionales? Explique qué tipo de modelo sería más adecuado para cada caso de uso y por qué.

| Tipo de Modelo     | Características Principales                                                                 | Casos de Uso en Chatbots                                      | Ejemplos de Modelos     |
|--------------------|---------------------------------------------------------------------------------------------|---------------------------------------------------------------|--------------------------|
| **Encoder-only**   | Solo codifica texto de entrada. Bidireccional. No genera texto.                            | Clasificación de intenciones, análisis de sentimientos        | BERT, RoBERTa            |
| **Decoder-only**   | Genera texto palabra por palabra. Autoregresivo. Unidireccional.                            | Generación de respuestas, completado de texto                 | GPT, GPT-2, GPT-3        |
| **Encoder-Decoder**| Codifica la entrada y luego genera una salida basada en esa codificación.                  | Traducción, resumen, generación controlada de respuestas      | T5, BART, mT5            |

### ¿Qué tipo de modelo sería más adecuado para cada caso de uso y por qué?

- **Encoder-only**: Útil para comprender lo que dice el usuario, como en la clasificación de intenciones o detección de entidades. No sirve para generar respuestas por sí solo.
- **Decoder-only**: Ideal para generar texto fluido y coherente, como respuestas en un chatbot. Usado cuando la prioridad es la generación del lenguaje.
- **Encoder-Decoder**: Útil cuando se necesita transformar texto de una forma a otra, como en chatbots que resumen información, traducen, o responden con base en entradas complejas.

---

## 2. Explique el concepto de "temperatura" en la generación de texto con LLMs. ¿Cómo afecta al comportamiento del chatbot y qué consideraciones debemos tener al ajustar este parámetro para diferentes aplicaciones?

## ¿Qué es la Temperatura en la Generación de Texto con LLMs?

La **temperatura** es un hiperparámetro que se utiliza durante la generación de texto con modelos de lenguaje como GPT para controlar la **aleatoriedad** de las predicciones.

### ¿Cómo Funciona?

Durante la generación, el modelo calcula una distribución de probabilidad sobre el siguiente token (palabra o subpalabra). La temperatura modifica esta distribución:

- Se aplica una fórmula como esta:  
P_i = exp(log(P_i) / T) / sum_j(exp(log(P_j) / T))

Donde `T` es la temperatura.

### Efecto de la Temperatura

- **Temperatura = 1.0** (valor por defecto):  
Comportamiento **estándar** del modelo.

- **Temperatura < 1.0** (por ejemplo, 0.2 a 0.7):  
La distribución se **agudiza** → el modelo es **más conservador y repetitivo**.  
Útil cuando se necesita **respuestas precisas, coherentes y controladas**.

- **Temperatura > 1.0** (por ejemplo, 1.2 o más):  
La distribución se **aplana** → el modelo es **más creativo, impredecible y variado**, pero puede volverse incoherente.  
Útil para tareas como escritura creativa o brainstorming.

### Ejemplos de Aplicación

| Aplicación                     | Temperatura Recomendada | Justificación                                               |
|-------------------------------|--------------------------|-------------------------------------------------------------|
| Chatbot para atención médica  | 0.3 - 0.5                | Respuestas seguras, formales, coherentes                    |
| Asistente educativo            | 0.5 - 0.7                | Cierta flexibilidad, pero con precisión en las explicaciones|
| Generador de historias        | 1.0 - 1.3                | Mayor creatividad y diversidad de narrativas                |
| Chat informal o roleplay      | 0.8 - 1.2                | Conversaciones variadas, espontáneas y menos rígidas        |

### Consideraciones al Ajustar la Temperatura

- No existe un valor único ideal: **depende del contexto de uso**.
- Temperaturas muy altas pueden generar incoherencias o errores.
- Se puede combinar con otros parámetros como `top_k` o `top_p` para refinar aún más el control del texto generado.

---

**Conclusión:**  
La temperatura es clave para modular el comportamiento del chatbot. Ajustarla correctamente puede marcar la diferencia entre una respuesta aburrida y repetitiva o una conversación natural y efectiva.




## 3. Describa las técnicas principales para reducir el problema de "alucinaciones" en chatbots basados en LLMs. ¿Qué estrategias podemos implementar a nivel de inferencia y a nivel de prompt engineering para mejorar la precisión factual de las respuestas?

## Reducción de Alucinaciones en Chatbots Basados en LLMs

Las **alucinaciones** son respuestas generadas por modelos de lenguaje que parecen correctas pero que son **fácticamente incorrectas o inventadas**. Este es uno de los desafíos principales en el uso de LLMs en aplicaciones críticas como educación, medicina o derecho.

---

### Técnicas para Reducir Alucinaciones

#### 🧠 A. A Nivel de Inferencia

1. **Temperatura baja**
   - Usar valores de temperatura entre `0.2` y `0.5` reduce la creatividad del modelo, promoviendo respuestas más seguras y precisas.

2. **Restricción de tokens (top-k / top-p sampling)**
   - Limitar la selección de tokens a los más probables (`top_k`) o acumulando una probabilidad límite (`top_p`) ayuda a evitar resultados improbables o aleatorios.

3. **Consulta iterativa y verificación**
   - Generar respuestas en múltiples pasos o pedir al modelo que **verifique su respuesta** antes de responder definitivamente.

4. **Recuperación basada en documentos (RAG)**
   - Combinar el LLM con un sistema de recuperación de documentos externos (como una base de datos o motor de búsqueda) para responder con información verificada.

---

#### ✍️ B. A Nivel de Prompt Engineering

1. **Instrucciones claras y específicas**
   - Instrucciones como _“Responde solo si estás seguro”_ o _“Si no sabes la respuesta, responde ‘No lo sé’”_ pueden reducir invenciones.

2. **Few-shot prompting con ejemplos reales**
   - Incluir ejemplos correctos y bien estructurados en el prompt para establecer un patrón deseado.

3. **Uso de cadenas de pensamiento ("chain-of-thought")**
   - Pedir al modelo que **razone paso a paso** puede mejorar la precisión lógica y reducir errores fácticos.

4. **Reforzar con roles o contexto de confiabilidad**
   - Frases como _“Eres un experto en medicina. Responde con hechos basados en evidencia científica”_ pueden guiar al modelo hacia mayor rigor.

5. **Incorporación de fuentes o referencias**
   - Pedir al modelo que cite una fuente o diga de dónde proviene la información:  
     _“Incluye la fuente de cada dato que menciones”_.

---

### Ejemplo de Prompt Mejorado

```plaintext
Eres un asistente legal que solo responde con información verificada. Si no tienes certeza, responde: "No tengo suficiente información para responder con precisión".

Pregunta: ¿Cuál es la edad mínima para votar en Argentina?

Conclusión
Reducir las alucinaciones en LLMs requiere una combinación de ajustes técnicos e ingeniería de prompts. No existe una solución única, pero aplicar varias de estas estrategias en conjunto puede mejorar considerablemente la precisión factual del chatbot.


